# DSC 106 Midterm Project

In [1]:
#import packages
import spotipy
import spotipy.util as util
import pandas as pd
import urllib.request

In [2]:
#Spotify API authentication token. The first parameter is the Spotify user id.
token = util.prompt_for_user_token('b5jfei3kyhlyuiadni4zvzl84',
                                   client_id = 'bee2dc948f6b4124830cd17f29fcf864',
                                   client_secret = '9ff98c1bb35345fcb051369450c295f6',
                                   redirect_uri = 'http://localhost:8000/callback')
#access the Spotify API
sp = spotipy.Spotify(auth=token)

In [3]:
#access the training playlist we made
training_playlist = sp.user_playlist('biggurlandyouknowit', 'spotify:playlist:1ZouLC1keHDTVeXRQDx6dq')

In [4]:
#read in the track info inside our training playlist
tracks_df = pd.DataFrame()
for track_data in training_playlist['tracks']['items']:
    track_df = pd.DataFrame(pd.DataFrame(dict([(k,pd.Series(v)) for k,v in track_data.items()]))['track']).transpose()
    tracks_df = tracks_df.append(track_df)
tracks_df = tracks_df.drop(tracks_df.columns[0], axis = 1)
tracks_df

,album,artists,available_markets,disc_number,duration_ms,episode,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track,track_number,type,uri,url
track,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,249293,False,False,{'isrc': 'USPR38619998'},{'spotify': 'https://open.spotify.com/track/37...,https://api.spotify.com/v1/tracks/37ZJ0p5Jm13J...,37ZJ0p5Jm13JPevGcx4SkF,False,Livin' On A Prayer,83,None,True,3,track,spotify:track:37ZJ0p5Jm13JPevGcx4SkF,NaN
track,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, US]",1,215933,False,False,{'isrc': 'GBCEE8000027'},{'spotify': 'https://open.spotify.com/track/5H...,https://api.spotify.com/v1/tracks/5HkFTCxSeJ3k...,5HkFTCxSeJ3kGNyQJbT4rJ,False,Another One Bites The Dust,48,None,True,3,track,spotify:track:5HkFTCxSeJ3kGNyQJbT4rJ,NaN
track,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,273480,False,False,{'isrc': 'USGF18714801'},{'spotify': 'https://open.spotify.com/track/0G...,https://api.spotify.com/v1/tracks/0G21yYKMZoHa...,0G21yYKMZoHa30cYVi1iA8,False,Welcome To The Jungle,72,None,True,1,track,spotify:track:0G21yYKMZoHa30cYVi1iA8,NaN
track,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, DK, DZ, FI, FR, GB, KW, MA, MX, NO, PL, P...",2,263040,False,False,{'isrc': 'GBAAA8500001'},{'spotify': 'https://open.spotify.com/track/3f...,https://api.spotify.com/v1/tracks/3fH4KjXFYMml...,3fH4KjXFYMmljxrcGrbPj9,False,Don't You (Forget About Me),75,None,True,1,track,spotify:track:3fH4KjXFYMmljxrcGrbPj9,NaN
track,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,239560,False,False,{'isrc': 'USWB18300001'},{'spotify': 'https://open.spotify.com/track/6F...,https://api.spotify.com/v1/tracks/6Fba9RZtC6vT...,6Fba9RZtC6vTY814JToDtP,False,Jump,58,https://p.scdn.co/mp3-preview/5c4fca39cd964267...,True,7,track,spotify:track:6Fba9RZtC6vTY814JToDtP,NaN
track,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, US]",1,355400,False,False,{'isrc': 'GBCEE0500364'},{'spotify': 'https://open.spotify.com/track/6l...,https://api.spotify.com/v1/tracks/6l8GvAyoUZwW...,6l8GvAyoUZwWDgF1e4822w,False,Bohemian Rhapsody,61,None,True,9,track,spotify:track:6l8GvAyoUZwWDgF1e4822w,NaN
track,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,266960,False,False,{'isrc': 'USSM10011897'},{'spotify': 'https://open.spotify.com/track/5M...,https://api.spotify.com/v1/tracks/5MxNLUsfh7uz...,5MxNLUsfh7uzROypsoO5qe,False,Dream On,78,https://p.scdn.co/mp3-preview/4c7fea33f70ea106...,True,3,track,spotify:track:5MxNLUsfh7uzROypsoO5qe,NaN
track,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,478173,False,False,{'isrc': 'USAT29900620'},{'spotify': 'https://open.spotify.com/track/51...,https://api.spotify.com/v1/tracks/51pQ7vY7WXzx...,51pQ7vY7WXzxskwloaeqyj,False,Stairway to Heaven - 1990 Remaster,69,https://p.scdn.co/mp3-preview/da57b9497b855355...,True,4,track,spotify:track:51pQ7vY7WXzxskwloaeqyj,NaN
track,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,254453,False,False,{'isrc': 'USWB19900178'},{'spotify': 'https://open.spotify.com/track/1l...,https://api.spotify.com/v1/tracks/1lbXEepatjRV...,1lbXEepatjRVjoG8pZMtdp,False,Dreams,62,https://p.scdn.co/mp3-preview/10b5068313e5ee28...,True,9,track,spotify:track:1lbXEepa

In [7]:
preview_urls = tracks_df[['name', 'preview_url']]
preview_urls

,name,preview_url
track,Livin' On A Prayer,None
track,Another One Bites The Dust,None
track,Welcome To The Jungle,None
track,Don't You (Forget About Me),None
track,Jump,https://p.scdn.co/mp3-preview/5c4fca39cd964267...
track,Bohemian Rhapsody,None
track,Dream On,https://p.scdn.co/mp3-preview/4c7fea33f70ea106...
track,Stairway to Heaven - 1990 Remaster,https://p.scdn.co/mp3-preview/da57b9497b855355...
track,Dreams,https://p.scdn.co/mp3-preview/10b5068313e5ee28...
track,The Long And Winding Road - Remastered 2015,None
